In [84]:
import scipy as sp
import numpy as np
import pandas as pd
import timeit
import re
import json
import pickle
import fastparquet
import os
os.chdir('/mnt/t48/bighomes-active/sfeng/patentdiffusion/')
seed = 3
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
import datetime
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.iolib.summary2 as summary2
import logging

## 1. Convert dataset to long for regression
- Code for include other variables from here: https://sfengc7.stern.nyu.edu:8888/notebooks/patentdiffusion/201808Results/Reg1016/1-NewDataReg.ipynb

In [74]:
ts = {}
ts1 = pd.read_pickle("DataStore/2018-07-P3/JTHReg0727/targ_cite_sim_reg_0727.pkl")
ts2 = pd.read_pickle("DataStore/2018-11/closest_nbr_control_1108.pkl")[["tp", "closest_pat", "tp_match_10", "closest_pat_match"]]
ts2 = ts2.rename(columns={"closest_pat": "cp", "closest_pat_match": "cp_match_10"})
ts["Sim"] = ts2
ts3 = pd.read_pickle("DataStore/2018-11/jth_rep_lawyers_control_1109.pkl")[["tp", "tp_match_10", "lawyer_cp", "lawyer_cp_match_10"]]
ts3 = ts3.rename(columns={"lawyer_cp": "cp", "lawyer_cp_match_10": "cp_match_10"})
ts["Lawyer"] = ts3

In [75]:
tsl = {}
for k,v in ts.items():
    tdf = v[["tp", "tp_match_10"]].rename(columns={"tp": "patent", "tp_match_10": "perc_match_10"})
    tdf["inv_msa_match"] = True
    cdf = v[["cp", "cp_match_10"]].rename(columns={"cp": "patent", "cp_match_10": "perc_match_10"})
    cdf["inv_msa_match"] = False
    tdf = tdf.append(cdf, ignore_index=True).reset_index(drop=True)
    
    tdf = tdf.dropna(how="any")
    tdf["patent"] = tdf["patent"].astype(int)
    tsl[k] = tdf
    del(tdf)
    
# Add original replication
ts1["inv_msa_match"] = ts1["inv_msa_match"].astype(bool)
ts1 = ts1.loc[ts1["perc_match_10"].notnull()]
tsl["PC"] = ts1[["patent", "perc_match_10", "inv_msa_match"]]

### Add fixed effects and normalize

In [76]:
pats_used = tsl["Sim"]["patent"].tolist() + tsl["Lawyer"]["patent"].tolist() + tsl["PC"]["patent"].tolist()
pats_used = np.unique(pats_used)

# Patent examiner
pe = fastparquet.ParquetFile("RawData/Cleaned/patexaminer1016.parq").to_pandas()

# Patent lawyers
ld = pd.read_csv("RawData/Cleaned/patent_lawyer.csv", index_col=0).drop_duplicates("patent")

# Top values
topn = {}
topn["examiner"] = re["examiner_id"].value_counts()[:100].index.tolist()
topn["lawyer"] = ld["lawyer_id"].value_counts()[:100].index.tolist()

# Use only what's in data
pe = pe.loc[pe["patent_id"].isin(pats_used)]
ld = ld.loc[ld["patent"].isin(pats_used)]

pe = dict(zip(pe["patent_id"], pe["examiner_id"]))
ld = dict(zip(ld["patent"], ld["lawyer_id"]))

# Top MSAs & Primary classes
pdf = fastparquet.ParquetFile("RawData/Cleaned/patent_loc_unique_us_0628.parq").to_pandas(["patent", "gyear", "primclass", "inv_msa"])
dup_pats = pd.read_pickle("RawData/Cleaned/duplicate_pattext_0712.pkl").tolist()
# Get relevant US Patents
pdf = pdf.loc[~pdf["patent"].isin(dup_pats)]

topn["inv_msa"] = pdf["inv_msa"].value_counts()[:100].index.tolist()
topn["primclass"] = pdf["primclass"].value_counts()[:100].index.tolist()

/homedir/eco/sfeng/bigdata/python/miniconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [77]:
# Scaler: Normalize on JTH control and target
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
tsl["PC"] = tsl["PC"].loc[tsl["PC"]["perc_match_10"].notnull()]
print(len(tsl["PC"]))
scaler.fit(tsl["PC"]["perc_match_10"].values.reshape(-1,1))

# Get year group
def get_year_group_10(x):
    if x in range(1975,1985):
        yg = "1975-85"
    elif x in range(1985,1995):
        yg = "1985-95"
    elif x in range(1995, 2005):
        yg = "1995-05"
    elif x in range(2005,2015):
        yg = "2005-15"
    else:
        yg = np.nan
    return yg

for k,v in tsl.items():
    rs = v
    rs["tp_gyear"] = rs["patent"].map(dict(zip(pdf["patent"], pdf["gyear"])))
    rs["tp_inv_msa"] = rs["patent"].map(dict(zip(pdf["patent"], pdf["inv_msa"])))
    rs["tp_primclass"] = rs["patent"].map(dict(zip(pdf["patent"], pdf["primclass"])))
    rs["tp_examiner"] = rs["patent"].map(pe)
    rs["tp_lawyer"] = rs["patent"].map(ld)
    
    for c in ["primclass", "inv_msa", "examiner", "lawyer"]:
        rs["tp_{0}_FE".format(c)] = rs["tp_{0}".format(c)].astype(str)
        rs.loc[~(rs["tp_{0}".format(c)].isin(topn[c])), "tp_{0}_FE".format(c)] = "other"

        print(rs["tp_{0}_FE".format(c)].value_counts()[:10])
        # Drop original
        try:
            rs = rs.drop("tp_{0}".format(c), 1)
        except Exception:
            pass
    
    # Normalize perc cite match
    rs = rs.loc[rs["perc_match_10"].notnull()]
    npm = scaler.transform(rs["perc_match_10"].values.reshape(-1,1))
    rs["norm_perc_match_10"] = npm
    
    # Add year group
    rs["year_group"] = rs["tp_gyear"].apply(get_year_group_10)
    tsl[k] = rs

INFO:root:507870
INFO:root:other    13746
514.0     7543
370.0     6470
438.0     5762
435.0     5705
424.0     5174
709.0     5124
455.0     4934
428.0     4833
600.0     4704
Name: tp_primclass_FE, dtype: int64
INFO:root:other                                                 52674
San Jose-Sunnyvale-Santa Clara, CA                    24057
New York-Northern New Jersey-Long Island, NY-NJ-PA    21487
Los Angeles-Long Beach-Santa Ana, CA                  16811
San Francisco-Oakland-Fremont, CA                     15909
Boston-Cambridge-Quincy, MA-NH                        13337
Chicago-Joliet-Naperville, IL-IN-WI                   12742
Detroit-Warren-Livonia, MI                             7925
Minneapolis-St. Paul-Bloomington, MN-WI                7836
Philadelphia-Camden-Wilmington, PA-NJ-DE-MD            7662
Name: tp_inv_msa_FE, dtype: int64
INFO:root:other                                       299881
6de8dd1fabc379f2470e51f0d7613a78fb8add9e       535
5dacd224066974a5b4abfe1c143f754

In [78]:
del(pdf, pe, ld)

In [79]:
pickle.dump(tsl, open("DataStore/2018-11/jth_rep_control_long_dict_1112.pkl", "wb"))

In [80]:
# Preview of results
for k,v in tsl.items():
    print(k)
    display(v[["perc_match_10", "year_group", "inv_msa_match"]].groupby(["year_group", "inv_msa_match"]).mean())

INFO:root:Sim


perc_match_10
year_group inv_msa_match               
1975-85    False               0.047628
           True                0.091571
1985-95    False               0.040534
           True                0.097179
1995-05    False               0.053538
           True                0.109568
2005-15    False               0.062872
           True                0.129871

INFO:root:Lawyer


perc_match_10
year_group inv_msa_match               
1975-85    False               0.075001
           True                0.093784
1985-95    False               0.078928
           True                0.100434
1995-05    False               0.088992
           True                0.114550
2005-15    False               0.092595
           True                0.134609

INFO:root:PC


perc_match_10
year_group inv_msa_match               
1975-85    False               0.037690
           True                0.090860
1985-95    False               0.034797
           True                0.097149
1995-05    False               0.045208
           True                0.109797
2005-15    False               0.055446
           True                0.130653

## 2. Regression

### 2.1 Specifying models

In [49]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.basicConfig(level=logging.INFO)
logger.addHandler(logging.FileHandler('Logs/JTH_reg_{0}.log'.format(datetime.datetime.now().\
                                                            strftime("%Y-%m-%d"), 'a')))
print = logging.info
print('good day to you madam fiona')
print('started')
print(datetime.datetime.now())

INFO:root:good day to you madam fiona
INFO:root:started
INFO:root:2018-11-12 20:09:23.751297


In [81]:
pathdir = "DataStore/2018-11/JTHReg1112/"
regs = {}
pc10 = {
    "PC FE-Year FE": "{0} ~ C(inv_msa_match) + C(tp_gyear) + C(tp_primclass_FE)",
    "MSA FE-Year FE": "{0} ~ C(inv_msa_match) + C(tp_gyear) + C(tp_primclass_FE) + C(tp_inv_msa_FE)",
    "Lawyer FE-Year FE": "{0} ~ C(inv_msa_match) + C(tp_gyear) + C(tp_primclass_FE)  + C(tp_inv_msa_FE)\
    + C(tp_lawyer_FE)",
    "Examiner FE-Year FE": "{0} ~ C(inv_msa_match) + C(tp_gyear) + C(tp_primclass_FE) + C(tp_inv_msa_FE)\
    + C(tp_lawyer_FE) + C(tp_examiner_FE)",
}

pc10_N = {"N "+k: "norm_"+v for k,v in pc10.items()}
pc10.update(pc10_N)
pc10 = {k: v.format("perc_match_10") for k,v in pc10.items()}

regs["JTH_model_names"] = pd.Series(list(pc10.keys())) 
regs["JTH_cite"] = pd.Series(list(pc10.values())) 

pickle.dump(regs, open(pathdir+"reg_models_112.pkl", "wb"))

In [82]:

info_dict = {'$N$':lambda x: "{0:d}".format(int(x.nobs)),
'Adjusted $R^2$':lambda x: "{:.2f}".format(x.rsquared_adj)}
def get_fit(formula, grouped_data, group_col, cov_type = "HC1", return_fit = False):
    summ = []
    tables = {}
    
    # If formula uses mean similarity, use grant year above 1980
    if "mean_sim_" in formula:
        grouped_data = grouped_data.loc[(grouped_data["tp_gyear"] >= 1980)]
    
    # Remove missing values used in formula
    col_used = re.findall('\((.*?)\)',formula)
    # Intersect with grouped_data columns
    col_used = list(set(col_used).intersection(set(list(grouped_data.columns))))
    print(col_used)
    
    grouped_data = grouped_data.dropna(how="any", subset=col_used).copy().reset_index(drop=True)
    
    # Get length of data
    print(("Length of data", len(grouped_data)))
                                    
    # Group and then get results
    grouped_data = grouped_data.groupby(group_col)
    
    for n,g in grouped_data:
        try:
            if cov_type == "HC1":
                fit = smf.ols(formula = formula, data = g, missing="drop").fit(cov_type="HC1")
            else:
                # Drop missing in the grouped data first
                # Cluster uses primary class so must have primary class FE
                cols_used2 = list(set(col_used).intersection(set(list(grouped_data.columns))))+["tp_primclass_FE"]
                g = g.dropna(subset=cols_used2, how="any").copy().reset_index(drop=True)
                fit = smf.ols(formula = formula, data = g).fit(cov_type="cluster",
                                                                              cov_kwds={'groups': g["tp_primclass_FE"]})
            # Get results tables
            tables[n] = fit.summary2().tables
            # Append results
            summ.append(fit)
        except Exception as e:
            print(n)
            logging.exception("Regression error")
            pass
    # Get full results output
    # Dataframe of full results
    res_no_stars = summary2.summary_col(summ, stars = False, \
    model_names = ["{0}".format(n) for n in tables.keys()],\
        info_dict = info_dict).tables[0]
    res_stars = summary2.summary_col(summ, stars = True, \
    model_names = ["{0}".format(n) for n in tables.keys()],\
        info_dict = info_dict).tables[0]
    
    # Get partial results
    # 1. Get relevant variables from index of full results: UPDATED
    regressors = [v for v in res_no_stars.index.unique() if ("sim_" in v) | ("match" in v) | ("common_" in v)]
    # 2. Make sure regressors come last
    regressors = regressors+["Intercept"]
    # 3. Get results with regressors
    part_res_no_stars = summary2.summary_col(summ, stars = False, \
    model_names = ["{0}".format(n) for n in tables.keys()],\
        info_dict = info_dict, regressor_order = regressors).tables[0]
    part_res_stars = summary2.summary_col(summ, stars = True, \
    model_names = ["{0}".format(n) for n in tables.keys()],\
        info_dict = info_dict, regressor_order = regressors).tables[0]
    
    # 4. Get index of where Intercept is and add 2 (to include standard error)
    last_ind = list(part_res_stars.index).index("Intercept")+2
    
    # 5. Get partial results
    part_res_no_stars = pd.concat([part_res_no_stars.iloc[:last_ind], part_res_no_stars.iloc[-2::]])
    part_res_stars = pd.concat([part_res_stars.iloc[:last_ind], part_res_stars.iloc[-2::]])
    
    if return_fit == True:
        return summ, tables, res_no_stars, res_stars, part_res_no_stars, part_res_stars
    else:
        return tables, res_no_stars, res_stars, part_res_no_stars, part_res_stars
    

In [94]:
tsl = pickle.load(open("DataStore/2018-11/jth_rep_control_long_dict_1112.pkl", "rb"))
for k,rs in tsl.items():
    print(k)
    cov = "HC1"
    samp_out = {}
    formulas = list(regs['JTH_cite'])
    formulas_ind = list(regs['JTH_cite'].index)
    for i, j in zip(formulas_ind, formulas):
        print((i, j))
        print(datetime.datetime.now())
        try:
            out = get_fit(j, rs, "year_group", cov, return_fit = False)
            samp_out[i] = {}
            samp_out[i]["model"] = j
            samp_out[i]["tables"] = out[0]
            samp_out[i]["res_no_stars"] = out[1]
            samp_out[i]["res_stars"] = out[2]
            samp_out[i]["part_res_no_stars"] = out[3]
            samp_out[i]["part_res_stars"] = out[4]
        except Exception as e:
            logging.exception("error here")
            pass
        print("finished")
        print(datetime.datetime.now())

    # Define outfile
    o_f = "JTH_Reg_{0}_out_1112.pkl".format(k)
    pickle.dump(samp_out, open(pathdir+o_f, "wb")) 

INFO:root:Sim
INFO:root:(0, 'perc_match_10 ~ C(inv_msa_match) + C(tp_gyear) + C(tp_primclass_FE)')
INFO:root:2018-11-13 11:31:10.791651
INFO:root:['tp_gyear', 'inv_msa_match', 'tp_primclass_FE']
INFO:root:('Length of data', 321438)
INFO:root:finished
INFO:root:2018-11-13 11:31:30.897516
INFO:root:(1, 'perc_match_10 ~ C(inv_msa_match) + C(tp_gyear) + C(tp_primclass_FE) + C(tp_inv_msa_FE)')
INFO:root:2018-11-13 11:31:30.900474
INFO:root:['tp_inv_msa_FE', 'tp_gyear', 'inv_msa_match', 'tp_primclass_FE']
INFO:root:('Length of data', 321438)
INFO:root:finished
INFO:root:2018-11-13 11:32:01.677093
INFO:root:(2, 'perc_match_10 ~ C(inv_msa_match) + C(tp_gyear) + C(tp_primclass_FE)  + C(tp_inv_msa_FE)    + C(tp_lawyer_FE)')
INFO:root:2018-11-13 11:32:01.679815
INFO:root:['tp_primclass_FE', 'tp_inv_msa_FE', 'tp_lawyer_FE', 'inv_msa_match', 'tp_gyear']
INFO:root:('Length of data', 321438)
INFO:root:finished
INFO:root:2018-11-13 11:32:41.166316
INFO:root:(3, 'perc_match_10 ~ C(inv_msa_match) + C(tp

In [98]:
pathdir = "DataStore/2018-11/JTHReg1112/"
res_out = pd.DataFrame()
for c in ["PC", "Sim", "Lawyer"]:
    o_f = "JTH_Reg_{0}_out_1112.pkl".format(c)
    res = pickle.load(open(pathdir+o_f, "rb"))
    for k in res.keys():
        lks = res[k]["model"].split(" ~ ")[0]

        # Selecting portion of results without intercept
        cdf = res[k]["part_res_stars"].reset_index()
        ic_ind = cdf.loc[cdf["index"] == "Intercept"].index[0]
        # Include N & R^2
        cdf = cdf.iloc[pd.np.r_[0:ic_ind,ic_ind+2:len(cdf)]]
        # Just include R^2
    #             cdf = cdf.iloc[pd.np.r_[0:ic_ind,ic_ind+2:len(cdf)-2, len(cdf)-1:len(cdf)]]
        cdf["Model"] = regs["JTH_model_names"][k]
        cdf["Model Num"] = k
        cdf["LKS"] = lks
        cdf["samp"] = "JTH Rep-"+c

        res_out = res_out.append(cdf)
        
res_out.to_csv(pathdir+"jth_rep_reg_1112.csv")